In [201]:
import pandas as pd
import numpy as np

# Setting basic values that will be used in calculations

In [202]:
trial_price = 4.99
subscription_fee = 29.99

cohorts = pd.DataFrame({
    'Cohort': ['Oct 2023', 'Nov 2023'],
    'Trial starts': [1250, 1173],
    '1st Month starts': [614, 612],
    '2nd Month starts': [325, None],
    '3rd Month starts': [None, None],
    '4th Month starts': [None, None],
    '5th Month starts': [None, None],
    '6th Month starts': [None, None],
})

old_retention_rates = {
    "to 0 Month": 1.00,
    "to 1 Month": 0.40,
    "to 2 Month": 0.60,
    "to 3 Month": 0.70,
    "to 4 Month": 0.80,
    "to 5 Month": 0.90,
    "to 6 Month": 0.95,
}

## filling NaN values with old_retention_rates * previous_month_value since there are not enough info about new retention values

In [203]:
for i in range(0, len(cohorts)):
    for j in range(1, len(cohorts.columns)):
        if pd.isna(cohorts.iloc[i, j]):
            cohorts.iloc[i, j] = np.int32(
                old_retention_rates[f"to {j - 1} Month"] * cohorts.iloc[i, j - 1]
            )
    
cohorts

,Cohort,Trial starts,1st Month starts,2nd Month starts,3rd Month starts,4th Month starts,5th Month starts,6th Month starts
0,Oct 2023,1250,614,325.0,227,181,162,153
1,Nov 2023,1173,612,367.0,256,204,183,173


## Calculating LTV for each cohort (oct 2023, nov 2023)

In [204]:
cohorts["LTV"] = (
        cohorts["Trial starts"] * trial_price + 
        cohorts.iloc[:, 2:].sum(axis=1) * subscription_fee
)

In [205]:
cohorts["LTV_per_user"] = cohorts["LTV"] / cohorts["Trial starts"]

In [206]:
cohorts_ltv = cohorts[["Cohort", "LTV", "LTV_per_user"]]
cohorts_ltv

,Cohort,LTV,LTV_per_user
0,Oct 2023,56080.88,44.864704
1,Nov 2023,59685.32,50.882626


## We can also take an average of those cohorts and users to get more accurate, global LTV

In [207]:
all_users_ltv = round(cohorts_ltv["LTV"].sum() / len(cohorts_ltv), 2)
per_user_ltv = round(all_users_ltv / (cohorts["Trial starts"].sum() / len(cohorts_ltv)), 2)
print(f"Average LTV (all users) {all_users_ltv} USD")
print(f"Average LTV per user {per_user_ltv} USD")

Average LTV (all users) 57883.1 USD
Average LTV per user 47.78 USD


# Conclusion:
### Each user who starts our new subscription model brings in approximately 47.78 USD in revenue.

#### However, it's important to note that since the trial now costs $4.99 instead of being free, fewer people may try our product, potentially leading to a lower profit with new model. To fully understand the impact, we need to compare the LTV of the old subscription model with that of the new model. Additionally, we need more precise data on new retention rates, as we currently have only a few months of data. For the missing data, we relied on retention rates from the old model, which may not be entirely accurate for the new one.

#### It would also be worth mentioning that employing machine learning, specifically linear regression on our retention rate data, could improve the accuracy of our predictions for how many users will start the new subscription model in the coming months (assuming the ML model is well-trained).
